In [3]:
import OOC
import string
import random
import numpy as np

In [4]:
def create_graph(X,Y):
    """
    Returns a graph of dimensions X and Y with random weights. 
    """
    #tab_vertices[X][Y]
    
    tab_vertices = [[0 for x in range(Y)] for y in range(X)] 
    new_graph = OOC.Graph()
    new_graph.dim_x = X
    new_graph.dim_y = Y 
    
    #creating the vertices
    for i in range (X):
        for j in range (Y):
            tab_vertices[i][j] = OOC.Vertice(str(i)+str(j))
            new_graph.addVertice(tab_vertices[i][j])
            
    # creating horizontal edges
    for i in range (X):
        for j in range (Y-1):
            weight = random.randint(0,5)
            new_graph.addEdge(OOC.Edge(tab_vertices[i][j],tab_vertices[i][j+1],weight))
            
    # creating vertical edges
    for j in range (Y):
        for i in range (X-1):
            weight = random.randint(0,5)
            new_graph.addEdge(OOC.Edge(tab_vertices[i][j],tab_vertices[i+1][j],weight))
            
    return new_graph

In [5]:
def create_graph_file(graph, filename):
    """
    Parameters : 
    - The graph that we want to save the information in a file.
    
    Returns a file containing all the information of the graph given in parameters.
    """
    with open(filename, mode='w+', encoding='utf8') as f:
    
        f.write("#rs "+str(graph.dim_x)+" cs "+ str(graph.dim_y) + "\n")
        f.write(str(len(graph.vertices)) + " " + str(len(graph.edges)) + "\n")
        
        f.write("val sommets\n")
        for vertice in graph.vertices:
            f.write(vertice.name + " 1" +"\n")
        
        f.write("arcs values\n")
        for edge in graph.edges:
            f.write(edge.x.name + " " + edge.y.name + " " + str(edge.weight)+ "\n")

In [6]:
def fill_graph_with_file(filename):

    """
    Parameters :
    - The name of the file that contains all the information to produce a given graph. The file must hast a particular structure
    
    Returns a graph filled with the information available in the file given in parameters.
    """
    new_graph = OOC.Graph() #Graph to fill
    
    with open(filename, mode = "r", encoding = "utf8") as f :
        lines = f.read().splitlines() #list of string containing all lines of the file

        #1st line : get nb columns/nb rows of the graph
        rs_cs = lines[0]
        rs_cs = rs_cs.replace("#rs","").replace("cs"," ")
        new_graph.dim_x, new_graph.dim_y = [int(x) for x in rs_cs.split()]

        #2nd line : get nb vertices/nb edges
        nb_v_e = lines[1]
        nb_vertices, nb_edges = [int(x) for x in nb_v_e.split()]

        # Check consistency with previous data
        m = new_graph.dim_x
        n = new_graph.dim_y
        if nb_vertices != m*n:
            print('Incoherent nb vertices and dimensions')
            return
        
        if nb_edges != (m-1)*n +(n-1)*m:
            print("Incoherent nb edges and dimensions")
            return
        
        #3rd line : get interesting indexes
        index1 = lines.index("val sommets")
        index2 = lines.index("arcs values")

        #get "val sommets" : name of vertices
        vals_sommets = [x.split() for x in lines[index1+1:index2]]  
        new_graph.vertices.extend([OOC.Vertice(sommet[0]) for sommet in vals_sommets])

        #get "arcs values" : edges
        arcs_values = [x.split() for x in lines[index2+1:]]
        new_graph.edges.extend([OOC.Edge(OOC.Vertice(edge[0]),OOC.Vertice(edge[1]),int(edge[2])) for edge in arcs_values])
        
        # Check consistency with previous data
        if nb_vertices != len(new_graph.vertices) or nb_edges != len(new_graph.edges):
            print('Incoherent X or Y dimensions')
            return
        
        return new_graph

In [7]:
def divide_blocks(Graph, nrows, ncols):

    """
    Parameters : 
    - Graph to cut
    - size of the blocks (nb of rows and cols)
    
    Returns a list of graphs. The graphs are subgraphs/blocks of the initial graph given in parameters.
    The cut attribute (graph.cut) contains a list of all adjacent edges.
    
    """
    
    sub_graphs = [] # list containing all the blocs (sub graphs)
    Graph.vertices = np.array(Graph.vertices)
    B = np.reshape(Graph.vertices, (-1, Graph.dim_y))    

    h, w = B.shape
    C = B.reshape(h//nrows, nrows, -1, ncols).swapaxes(1,2).reshape(-1, nrows, ncols)
    
    print("B =\n",B) # "grid" initial graph
    print("C =\n",C) # "grid" subgraphs/blocks
    
    for block in C:
        new_graph = OOC.Graph()
        new_graph.dim_x = nrows
        new_graph.dim_y = ncols
        
        for line in block:
            for vertice in line:
                new_graph.addVertice(vertice) # add each element to the list of vertices
                sg_edges,sg_cut_edges = Graph.adjacentEdge(vertice, new_graph.vertices)
                
                new_graph.edges.extend(sg_edges)
                new_graph.cut.extend(sg_cut_edges)
                
        new_graph.edges = list(set (new_graph.edges))
        new_graph.cut = set(new_graph.cut).difference(new_graph.edges)
        sub_graphs.append(new_graph) 
           
    #print(sub_graphs[1].edges)
    #print("\n",sub_graphs[1].cut)
    return sub_graphs
    

In [12]:
         
def main():

    graph1 = create_graph(6,6) #create a random graph
    
    print("----- PRINT GRAPH 1 -----")
    graph1.affiche()
    
    create_graph_file(graph1, "file.graph") #save it in a file
    
    
    graph2 = fill_graph_with_file("file.graph") #read this file save it in another graph
    print("----- PRINT GRAPH 2 -----")
    graph2.affiche() # should have graph1 = graph2

    print("----- DIVIDE BLOCKS -----")
    divide_blocks(graph1,3,2)  

if __name__ == '__main__':
    main()
    

----- PRINT GRAPH 1 -----
Vertices :
[ 00 01 02 03 04 05 10 11 12 13 14 15 20 21 22 23 24 25 30 31 32 33 34 35 40 41 42 43 44 45 50 51 52 53 54 55 ]
Edges : 
[ (00, 01)(w = 5)
 (01, 02)(w = 1)
 (02, 03)(w = 1)
 (03, 04)(w = 4)
 (04, 05)(w = 5)
 (10, 11)(w = 3)
 (11, 12)(w = 3)
 (12, 13)(w = 2)
 (13, 14)(w = 0)
 (14, 15)(w = 2)
 (20, 21)(w = 3)
 (21, 22)(w = 1)
 (22, 23)(w = 0)
 (23, 24)(w = 3)
 (24, 25)(w = 0)
 (30, 31)(w = 2)
 (31, 32)(w = 2)
 (32, 33)(w = 2)
 (33, 34)(w = 2)
 (34, 35)(w = 4)
 (40, 41)(w = 3)
 (41, 42)(w = 5)
 (42, 43)(w = 3)
 (43, 44)(w = 3)
 (44, 45)(w = 1)
 (50, 51)(w = 1)
 (51, 52)(w = 1)
 (52, 53)(w = 0)
 (53, 54)(w = 4)
 (54, 55)(w = 3)
 (00, 10)(w = 0)
 (10, 20)(w = 3)
 (20, 30)(w = 2)
 (30, 40)(w = 2)
 (40, 50)(w = 1)
 (01, 11)(w = 0)
 (11, 21)(w = 5)
 (21, 31)(w = 1)
 (31, 41)(w = 5)
 (41, 51)(w = 3)
 (02, 12)(w = 3)
 (12, 22)(w = 2)
 (22, 32)(w = 1)
 (32, 42)(w = 1)
 (42, 52)(w = 4)
 (03, 13)(w = 3)
 (13, 23)(w = 3)
 (23, 33)(w = 5)
 (33, 43)(w = 1)
 (43, 53

#rs 3 cs 4
12 17
val sommets
0 1
1 1
2 1
3 1
4 1
5 1
6 1
7 1
8 1
9 1
10 1
11 1
arcs values
0 1 3
1 2 1
2 3 2
4 5 2
5 6 5
6 7 1
8 9 7
9 10 8
10 11 3
0 4 3
4 8 4
1 5 10
5 9 1
2 6 0
6 10 0
3 7 0
7 11 10